<h2 style="color:#696969">Housing, Health, and Happines <br> Creative extension </h2> 
<h1 style="text-align:center">Effect of different matching methods on the paper's conclusions</h1>
<p style="text-align:justify"><strong>“Housing, Health, and Happiness”</strong> is an extremely interesting paper which demonstrates that simple housing improvements in slum areas are beneficial for child health and cognitive development as well as adult happiness. If this is indeed true, the paper should be a source of inspiration for governments taking action to combat poverty. However, this pleasing conclusion is drawn from an observational study and may thus be biased by confounders. For this reason, we decided to see if we obtain <strong>the same conclusion</strong> after having <strong>properly balanced the control and treatment group</strong> using adequate matching methods. To do so, we are going to use the same dataset and the same statistical methods (linear regressions) as the paper, but extra efforts will be made to find the best matching between the control and treatment groups.</p>
    


<p> We decided to find the best balance for covariates that can be grouped into 5 <strong>household related categories</strong>. In our view, the chosen covariates are unrelated to the treatment and are therefore good candidates for balancing the groups. The 5 categories and corresponding covariates are the following:</p>
<br>
<strong> Demographic covariates (13):</strong>
<ul>
    <li>S_HHpeople: 2005 Survey - Number of household members</li>
    <li>S_headage: 2005 Survey - Head of household's age</li>
    <li>S_spouseage: 2005 Survey - Spouse's age</li>
    <li>S_headeduc 2005 Survey - Head of household's years of schooling</li>
    <li>S_spouseeduc 2005 Survey - Spouse's years of schooling</li>
    <li>S_dem1: 2005 Survey - Proportion of Males 0-5yrs in household</li>
    <li>S_dem2: 2005 Survey - Proportion of Males 6-17yrs in household</li>
    <li>S_dem3: 2005 Survey - Proportion of Males 18-49yrs in household</li>
    <li>S_dem4: 2005 Survey - Proportion of Males 50+yrs in household</li>
    <li>S_dem5: 2005 Survey - Proportion of Females 0-5yrs in household</li>
    <li>S_dem6: 2005 Survey - Proportion of Females 6-17yrs in household</li>
    <li>S_dem7: 2005 Survey - Proportion of Females 18-49yrs in household</li>
    <li>S_dem8: 2005 Survey - Proportion of Females 50+yrs in household</li>
</ul>

<strong> Housing covariates (4):</strong>
<ul>
    <li>S_rooms: 2005 Survey - Number of rooms</li>
    <li>S_waterland: 2005 Survey - Water connection (=1)</li>
    <li>S_waterhouse: 2005 Survey - Water connection inside the house (=1)</li>
    <li>S_electricity 2005 Survey - Electricity (=1)</li>
</ul>
<strong> Economic covariates (5):</strong>
<ul>
    <li>S_incomepc: 2005 Survey - Total household income per capita</li>
    <li>S_assetspc: 2005 Survey - Total value of household assets per capita</li>
    <li>S_shpeoplework: 2005 Survey - Proportion of household members who work</li>
    <li>S_microenter: 2005 Survey - Household operates a microenterprise (=1)</li>
    <li>S_hrsworkedpc: 2005 Survey - Hours worked by household members per capita</li>
</ul>
<strong> Hygienic covariates (4):</strong>
<ul>
    <li>S_hasanimals: 2005 Survey - Household has animals on land (=1)</li>
    <li>S_animalsinside: 2005 Survey - Animals allowed to enter the house (=1)</li>
    <li>S_garbage: 2005 Survey - Uses garbage collection service (=1)</li>
    <li>S_washhands: 2005 Survey - Number of times respondent washed hands the day before</li>
</ul>
<strong> Social assistance covariates (3):</strong>
<ul>
    <li>S_cashtransfers: 2005 Survey - Transfers per capita from government programs</li>
    <li>S_milkprogram: 2005 Survey - Household beneficiary of government milk supplement program (=1)</li>
    <li>S_foodprogram: 2005 Survey - Household beneficiary of government food program (=1)</li>
</ul>